In [6]:
import os
import json
from langsmith import Client
from dotenv import load_dotenv

# 1. 환경 변수 로드: api.env 파일에서 LANGSMITH_API_KEY를 읽어옵니다.
load_dotenv('/Users/minjoo/Desktop/SeSac/final/api.env')
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
if not LANGSMITH_API_KEY:
    raise ValueError("LANGSMITH_API_KEY를 찾을 수 없습니다. api.env 파일을 확인하세요.")

# 2. Langsmith 클라이언트 초기화 및 프롬프트 가져오기
client = Client(api_key=LANGSMITH_API_KEY)
prompt_data = client.pull_prompt("langchain-ai/rag-fusion-query-generation", include_model=True)
print("불러온 prompt_data:", prompt_data)

# 3. JSON 파일 로드: rag fusion에 사용할 데이터 (문서 목록 등)
json_path = "/Users/minjoo/Desktop/SeSac/final/20250304.json"
with open(json_path, "r", encoding="utf-8") as f:
    documents = json.load(f)
print(f"로드된 문서 개수: {len(documents)}")

# 4. 랭킹 함수 수정: JSON의 "서비스목적요약"과 "지원내용" 필드를 사용하여 점수 산정
def rank_documents(query, docs):
    query_terms = set(query.lower().split())
    ranked_docs = []
    for doc in docs:
        # None 체크: 값이 None이면 빈 문자열("")로 대체
        service_summary = doc.get("서비스목적요약") or ""
        support_content = doc.get("지원내용") or ""
        doc_text = (service_summary + " " + support_content).lower()
        score = sum(1 for term in query_terms if term in doc_text)
        ranked_docs.append((score, doc))
    ranked_docs.sort(key=lambda x: x[0], reverse=True)
    return ranked_docs



# 5. 재정렬(rerank) 함수: prompt_data에 rerank 메서드가 있다면 이를 사용하고,
# 없으면 기존 rank_documents 함수를 사용하도록 합니다.
def rerank_documents(query, docs, prompt_data):
    if hasattr(prompt_data, 'rerank'):
        # 예: prompt_data.rerank(query, docs)  (실제 API에 따라 인자와 반환값이 달라질 수 있음)
        return prompt_data.rerank(query, docs)
    else:
        return rank_documents(query, docs)

# 6. RAG Fusion 함수: 상위 문서를 결합하여 최종 프롬프트 생성
def rag_fusion(query, docs, prompt_data):
    # 여기서는 rerank_documents 함수를 이용해 문서를 재정렬합니다.
    ranked_docs = rerank_documents(query, docs, prompt_data)
    # 상위 3개 문서 선택 (필요시 조정)
    top_docs = [doc for score, doc in ranked_docs[:15]]
    docs_text = "\n".join(
        (((doc.get("서비스목적요약") or "") + " " + (doc.get("지원내용") or "")).strip())
        for doc in top_docs
    )
    final_prompt = f"Query: {query}\n\nDocuments:\n{docs_text}\n\nAnswer:"
    return final_prompt

# 7. 사용자 쿼리 설정 (실제 사용할 query 문구로 변경)
query = "서울특별시 도봉구에 사는 34세 청년 남자에게 맞는 서비스를 찾아줘."
final_prompt = rag_fusion(query, documents, prompt_data)
print("\n최종 프롬프트:")
print(final_prompt)


불러온 prompt_data: input_variables=['original_query'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'rag-fusion-query-generation', 'lc_hub_commit_hash': '478b448e096b977446865108fad34282e6e1a84ae8b8540572ed0df238229a11'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that generates multiple search queries based on a single input query.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], input_types={}, partial_variables={}, template='Generate multiple search queries related to: {original_query}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='OUTPUT (4 queries):'), additional_kwargs={})]
로드된 문서 개수: 10232

최종 프롬프트:
Query: 서울특별시 도봉구에 사는 34세 청년 남자에게 맞는 서비스를 찾아줘.

Documents:
필요에 맞게 기본서비스(

In [7]:
ranked_docs = rerank_documents(query, documents, prompt_data)
top_docs = [doc for score, doc in ranked_docs[:15]]
print("상위 3개 문서 개수:", len(top_docs))
for i, doc in enumerate(top_docs, 1):
    service_summary = doc.get("서비스목적요약") or ""
    support_content = doc.get("지원내용") or ""
    print(f"\n문서 {i} 길이:", len(service_summary + " " + support_content))


상위 3개 문서 개수: 15

문서 1 길이: 847

문서 2 길이: 678

문서 3 길이: 273

문서 4 길이: 105

문서 5 길이: 341

문서 6 길이: 120

문서 7 길이: 610

문서 8 길이: 89

문서 9 길이: 473

문서 10 길이: 71

문서 11 길이: 1203

문서 12 길이: 168

문서 13 길이: 175

문서 14 길이: 53

문서 15 길이: 786


In [8]:
import os
import json
from langsmith import Client
from dotenv import load_dotenv

# 1. 환경 변수 로드: api.env 파일에서 LANGSMITH_API_KEY를 읽어옵니다.
load_dotenv('/Users/minjoo/Desktop/SeSac/final/api.env')
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
if not LANGSMITH_API_KEY:
    raise ValueError("LANGSMITH_API_KEY를 찾을 수 없습니다. api.env 파일을 확인하세요.")

# 2. Langsmith 클라이언트 초기화 및 프롬프트 가져오기
client = Client(api_key=LANGSMITH_API_KEY)
prompt_data = client.pull_prompt("langchain-ai/rag-fusion-query-generation", include_model=True)
print("불러온 prompt_data:", prompt_data)

# 3. JSON 파일 로드: rag fusion에 사용할 데이터 (문서 목록 등)
json_path = "/Users/minjoo/Desktop/SeSac/final/20250304.json"
with open(json_path, "r", encoding="utf-8") as f:
    documents = json.load(f)
print(f"로드된 문서 개수: {len(documents)}")

# 4. 기존의 rank_documents 함수 (단순 랭킹 방식)
def rank_documents(query, docs):
    query_terms = set(query.lower().split())
    ranked_docs = []
    for doc in docs:
        # '서비스목적요약'과 '지원내용' 필드 값이 None일 경우 빈 문자열로 대체
        service_summary = doc.get("서비스목적요약") or ""
        support_content = doc.get("지원내용") or ""
        doc_text = (service_summary + " " + support_content).lower()
        score = sum(1 for term in query_terms if term in doc_text)
        ranked_docs.append((score, doc))
    ranked_docs.sort(key=lambda x: x[0], reverse=True)
    return ranked_docs

# 5. 재정렬(rerank) 함수: prompt_data에 rerank 메서드가 있다면 이를 사용하고,
# 없으면 기존 rank_documents 함수를 사용하도록 합니다.
def rerank_documents(query, docs, prompt_data):
    if hasattr(prompt_data, 'rerank'):
        # 예시: prompt_data.rerank(query, docs)
        return prompt_data.rerank(query, docs)
    else:
        return rank_documents(query, docs)

# 6. RAG Fusion 함수: 상위 문서를 결합하여 최종 프롬프트 생성
def rag_fusion(query, docs, prompt_data):
    # rerank_documents 함수를 사용해 문서를 재정렬
    ranked_docs = rerank_documents(query, docs, prompt_data)
    # 상위 3개 문서 선택 (필요에 따라 조정)
    top_docs = [doc for score, doc in ranked_docs[:15]]
    # 각 문서의 "서비스ID"와 "서비스명", 그리고 "서비스목적요약", "지원내용"을 결합하여 텍스트 생성
    docs_text = "\n\n".join(
        f"서비스ID: {doc.get('서비스ID', 'N/A')}\n"
        f"서비스명: {doc.get('서비스명', 'N/A')}\n"
        f"{((doc.get('서비스목적요약') or '') + ' ' + (doc.get('지원내용') or '')).strip()}"
        for doc in top_docs
    )
    final_prompt = f"Query: {query}\n\nDocuments:\n{docs_text}\n\nAnswer:"
    return final_prompt

# 7. 사용자 쿼리 설정 (실제 사용할 query 문구로 변경)
query = "서울특별시 도봉구에 사는 34세 청년 남자에게 맞는 서비스를 찾아줘."
final_prompt = rag_fusion(query, documents, prompt_data)
print("\n최종 프롬프트:")
print(final_prompt)

불러온 prompt_data: input_variables=['original_query'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'langchain-ai', 'lc_hub_repo': 'rag-fusion-query-generation', 'lc_hub_commit_hash': '478b448e096b977446865108fad34282e6e1a84ae8b8540572ed0df238229a11'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant that generates multiple search queries based on a single input query.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], input_types={}, partial_variables={}, template='Generate multiple search queries related to: {original_query}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='OUTPUT (4 queries):'), additional_kwargs={})]
로드된 문서 개수: 10232

최종 프롬프트:
Query: 서울특별시 도봉구에 사는 34세 청년 남자에게 맞는 서비스를 찾아줘.

Documents:
서비스ID: 135200